In [6]:
%run prophet_funcs_v2.py
#initialization script creates functions and adds libraries to environment

In [9]:

tickerStrings = ['AAPL', 'MSFT']

#Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
period = '1mo'

vars = tickerStrings,period

#FetchData: This function takes the input vars and uses the yfinance package to download data from the yahoo finance website into a dataframe.
df = FetchData(vars)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,ticker
Date,,,,,,,
2021-10-04,141.759995,142.210007,138.270004,139.139999,139.139999,98322000,AAPL
2021-10-05,139.490005,142.240005,139.360001,141.110001,141.110001,80861100,AAPL
2021-10-06,139.470001,142.149994,138.369995,142.000000,142.000000,83221100,AAPL
2021-10-07,143.059998,144.220001,142.720001,143.289993,143.289993,61732700,AAPL
2021-10-08,144.029999,144.179993,142.559998,142.899994,142.899994,58718700,AAPL
2021-10-11,142.270004,144.809998,141.809998,142.809998,142.809998,64452200,AAPL
2021-10-12,143.229996,143.250000,141.039993,141.509995,141.509995,73035900,AAPL
2021-10-13,141.240005,141.399994,139.199997,140.910004,140.910004,78762700,AAPL
2021-10-14,142.110001,143.880005,141.509995,143.759995,143.759995,69907100,AAPL
